Polars lazy experiments

This was a WIP in 202311 and was abandoned, so ignore

In [1]:
import polars as pl
import datetime

%load_ext autoreload
%autoreload 2

import datetime, os
display(f"Polars {pl.__version__}")
display(f'Running: {datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")}, env {os.getenv("CONDA_DEFAULT_ENV")}')

%load_ext ipython_memory_usage

%imu_start


'Polars 0.19.17'

Enabling IPython Memory Usage, use %imu_start to begin, %imu_stop to end


'IPython Memory Usage started'

In [1] used 1.1 MiB RAM in 0.11s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 81.2 MiB


In [2]:
from pprint import pprint
pprint(pl.Config.save())

'{"environment":{},"direct":{"set_fmt_float":"mixed","set_float_precision":null,"set_thousands_separator":"","set_decimal_separator":".","set_trim_decimal_zeros":false}}'
In [2] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 81.2 MiB


In [24]:
pl.Config.set_verbose() # enable verbose logs

polars.config.Config

In [24] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 814.4 MiB


In [16]:
pl.threadpool_size()


16

In [16] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 792.1 MiB


In [3]:
dfpll = pl.scan_parquet("../test_result.parquet/*.parquet")
f"{dfpll.select(pl.count()).collect().item():,} items"


'639,506,968 items'

In [3] used 941.0 MiB RAM in 4.27s (system mean cpu 38%, single max cpu 100%), peaked 8855.3 MiB above final usage, current RAM usage now 1022.1 MiB


# Count only

In [8]:
dfpll.select(pl.count()).collect()

count
u32
639506968


In [8] used -425.0 MiB RAM in 4.08s (system mean cpu 38%, single max cpu 100%), peaked 9243.3 MiB above final usage, current RAM usage now 597.7 MiB


In [9]:
dfpll.select(pl.count()).collect(streaming=True)

count
u32
639506968


In [9] used 71.6 MiB RAM in 7.93s (system mean cpu 12%, single max cpu 100%), peaked 4394.2 MiB above final usage, current RAM usage now 669.2 MiB


In [4]:
print(dfpll.select(pl.count()).explain())

 SELECT [count()] FROM

    Parquet SCAN 854 files: first file: ../test_result.parquet/part.0.parquet
    PROJECT 1/14 COLUMNS
In [4] used 0.1 MiB RAM in 0.10s (system mean cpu 15%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1022.3 MiB


In [25]:
print(dfpll.select(pl.count()).explain(streaming=True))

 SELECT [count()] FROM
  --- STREAMING

  Parquet SCAN 854 files: first file: ../test_result.parquet/part.0.parquet
  PROJECT 1/14 COLUMNS  --- END STREAMING

    DF []; PROJECT */0 COLUMNS; SELECTION: "None"
In [25] used 0.6 MiB RAM in 0.12s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 815.0 MiB


/tmp/ipykernel_53334/1326765554.py:1: UserWarning: Cannot combine 'streaming' with 'comm_subplan_elim'. CSE will be turned off.
  print(dfpll.select(pl.count()).explain(streaming=True))
POLARS PREFETCH_SIZE: 32


# Filter and count

In [10]:
dfpll.filter(pl.col("test_date") > datetime.datetime(2021, 1, 1)).select(
    pl.count()
).collect()

count
u32
82012245


In [10] used 54.7 MiB RAM in 2.94s (system mean cpu 69%, single max cpu 100%), peaked 1928.0 MiB above final usage, current RAM usage now 723.9 MiB


In [11]:
dfpll.filter(pl.col("test_date") > datetime.datetime(2021, 1, 1)).select(
    pl.count()
).collect(streaming=True)

count
u32
82012245


In [11] used -28.7 MiB RAM in 7.78s (system mean cpu 14%, single max cpu 100%), peaked 871.8 MiB above final usage, current RAM usage now 695.2 MiB


In [6]:
# 82012876
print(dfpll.filter(pl.col("test_date") > datetime.datetime(2021, 1, 1)).select(
    pl.count()
).explain())

 SELECT [count()] FROM

    Parquet SCAN 854 files: first file: ../test_result.parquet/part.0.parquet
    PROJECT 2/14 COLUMNS
    SELECTION: [(col("test_date")) > (2021-01-01 00:00:00)]
In [6] used 0.1 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 1022.5 MiB


In [7]:
import datetime

# 82012876
print(dfpll.filter(pl.col("test_date") > datetime.datetime(2021, 1, 1)).select(
    pl.count()
).explain(streaming=True))

 SELECT [count()] FROM
  --- STREAMING

  Parquet SCAN 854 files: first file: ../test_result.parquet/part.0.parquet
  PROJECT 2/14 COLUMNS
  SELECTION: [(col("test_date")) > (2021-01-01 00:00:00)]  --- END STREAMING

    DF []; PROJECT */0 COLUMNS; SELECTION: "None"
In [7] used 0.1 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 1022.6 MiB


/tmp/ipykernel_53334/2477444838.py:6: UserWarning: Cannot combine 'streaming' with 'comm_subplan_elim'. CSE will be turned off.
  ).explain(streaming=True))


# Filter, sort, group by, agg

In [12]:
result_lz = (
    dfpll.filter(pl.col("test_date") > datetime.datetime(2018, 1, 1))
    .with_columns((pl.col("test_result") == pl.lit("P")).alias("passed"))
    .sort(pl.col("test_date"))
    .group_by_dynamic("test_date", every="1w")
    .agg(pl.col("passed").sum())
    .collect(streaming=False)
)

In [12] used 140.0 MiB RAM in 13.38s (system mean cpu 39%, single max cpu 100%), peaked 9345.8 MiB above final usage, current RAM usage now 835.2 MiB


In [13]:
result_lz = (
    dfpll.filter(pl.col("test_date") > datetime.datetime(2018, 1, 1))
    .with_columns((pl.col("test_result") == pl.lit("P")).alias("passed"))
    .sort(pl.col("test_date"))
    .group_by_dynamic("test_date", every="1w")
    .agg(pl.col("passed").sum())
    .collect(streaming=True)
)
# 17s + 9.5GB
#In [11] used -1.3 MiB RAM in 17.84s (system mean cpu 34%, single max cpu 100%), 
# peaked 9534.2 MiB above final usage, current RAM usage now 1022.9 MiB


In [13] used -43.1 MiB RAM in 17.88s (system mean cpu 35%, single max cpu 100%), peaked 9542.0 MiB above final usage, current RAM usage now 792.1 MiB


In [14]:
print(
    dfpll.filter(pl.col("test_date") > datetime.datetime(2018, 1, 1))
    .with_columns((pl.col("test_result") == pl.lit("P")).alias("passed"))
    .sort(pl.col("test_date"))
    .group_by_dynamic("test_date", every="1w")
    .agg(pl.col("passed").sum())
    .explain(streaming=False)
)

AGGREGATE
	[col("passed").sum()] BY [] FROM
  SORT BY [col("test_date")]
     WITH_COLUMNS:
     [[(col("test_result")) == (Utf8(P))].alias("passed")]

        Parquet SCAN 854 files: first file: ../test_result.parquet/part.0.parquet
        PROJECT 2/14 COLUMNS
        SELECTION: [(col("test_date")) > (2018-01-01 00:00:00)]
In [14] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 792.1 MiB


In [26]:
print(
    dfpll.filter(pl.col("test_date") > datetime.datetime(2018, 1, 1))
    .with_columns((pl.col("test_result") == pl.lit("P")).alias("passed"))
    .sort(pl.col("test_date"))
    .group_by_dynamic("test_date", every="1w")
    .agg(pl.col("passed").sum())
    .explain(streaming=True)
)

AGGREGATE
	[col("passed").sum()] BY [] FROM
  --- STREAMING
SORT BY [col("test_date")]
   WITH_COLUMNS:
   [[(col("test_result")) == (Utf8(P))].alias("passed")]

      Parquet SCAN 854 files: first file: ../test_result.parquet/part.0.parquet
      PROJECT 2/14 COLUMNS
      SELECTION: [(col("test_date")) > (2018-01-01 00:00:00)]  --- END STREAMING

    DF []; PROJECT */0 COLUMNS; SELECTION: "None"
In [26] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 815.0 MiB


/tmp/ipykernel_53334/4111795314.py:7: UserWarning: Cannot combine 'streaming' with 'comm_subplan_elim'. CSE will be turned off.
  .explain(streaming=True)
POLARS PREFETCH_SIZE: 32


In [17]:
pl.Config.set_streaming_chunk_size(100)
result_lz = (
    dfpll.filter(pl.col("test_date") > datetime.datetime(2018, 1, 1))
    .with_columns((pl.col("test_result") == pl.lit("P")).alias("passed"))
    .sort(pl.col("test_date"))
    .group_by_dynamic("test_date", every="1w")
    .agg(pl.col("passed").sum())
    .collect(streaming=True)
)


In [17] used 2864.5 MiB RAM in 27.14s (system mean cpu 34%, single max cpu 100%), peaked 9594.3 MiB above final usage, current RAM usage now 3656.6 MiB


In [18]:
pl.Config.set_streaming_chunk_size(1000)
result_lz = (
    dfpll.filter(pl.col("test_date") > datetime.datetime(2018, 1, 1))
    .with_columns((pl.col("test_result") == pl.lit("P")).alias("passed"))
    .sort(pl.col("test_date"))
    .group_by_dynamic("test_date", every="1w")
    .agg(pl.col("passed").sum())
    .collect(streaming=True)
)


In [18] used -2162.8 MiB RAM in 19.26s (system mean cpu 33%, single max cpu 100%), peaked 9433.4 MiB above final usage, current RAM usage now 1493.8 MiB


In [19]:
pl.Config.set_streaming_chunk_size(5000)
result_lz = (
    dfpll.filter(pl.col("test_date") > datetime.datetime(2018, 1, 1))
    .with_columns((pl.col("test_result") == pl.lit("P")).alias("passed"))
    .sort(pl.col("test_date"))
    .group_by_dynamic("test_date", every="1w")
    .agg(pl.col("passed").sum())
    .collect(streaming=True)
)


In [19] used -559.1 MiB RAM in 18.28s (system mean cpu 33%, single max cpu 100%), peaked 9884.0 MiB above final usage, current RAM usage now 934.8 MiB


In [20]:
pl.Config.set_streaming_chunk_size(25000)
result_lz = (
    dfpll.filter(pl.col("test_date") > datetime.datetime(2018, 1, 1))
    .with_columns((pl.col("test_result") == pl.lit("P")).alias("passed"))
    .sort(pl.col("test_date"))
    .group_by_dynamic("test_date", every="1w")
    .agg(pl.col("passed").sum())
    .collect(streaming=True)
)


In [20] used -155.0 MiB RAM in 18.10s (system mean cpu 32%, single max cpu 100%), peaked 9459.4 MiB above final usage, current RAM usage now 779.8 MiB


In [21]:
pl.Config.set_streaming_chunk_size(50000)
result_lz = (
    dfpll.filter(pl.col("test_date") > datetime.datetime(2018, 1, 1))
    .with_columns((pl.col("test_result") == pl.lit("P")).alias("passed"))
    .sort(pl.col("test_date"))
    .group_by_dynamic("test_date", every="1w")
    .agg(pl.col("passed").sum())
    .collect(streaming=True)
)


In [21] used 40.5 MiB RAM in 18.31s (system mean cpu 33%, single max cpu 100%), peaked 9391.4 MiB above final usage, current RAM usage now 820.3 MiB
